In [0]:
import    os
import    matplotlib.pyplot      as plt
import    cv2                    as cv
import    numpy                  as np
import    tensorflow             as tf

from   sklearn.decomposition     import  PCA
from   tqdm                      import  tqdm

from   sklearn.ensemble          import RandomForestClassifier
from   sklearn.model_selection   import cross_val_score
from   sklearn.model_selection   import KFold
from   sklearn.metrics           import *

In [2]:
pathImgTraining     =      '/content/drive/My Drive/DataSet/training/img/'
pathLblTraining     =      '/content/drive/My Drive/DataSet/training/labels/'
pathImgTest         =      '/content/drive/My Drive/DataSet/test/img/'
pathLblTest         =      '/content/drive/My Drive/DataSet/test/labels/'
pathUniverse        =       '/content/drive/My Drive/DataSet/universe/'

parchSize           =      40
parcheDes           =      40


model_VGG16         = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(parchSize,parchSize,3))
model_VGG16_conv1   = tf.keras.models.Model(inputs=model_VGG16.input, outputs=model_VGG16.get_layer('block1_conv1').output)
pca                 = PCA(n_components = 64)

W0821 00:36:56.788883 140021932177280 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


58892288/58889256 [==============================] - 2s 0us/step


In [3]:
imagenesTraining    =   os.listdir(pathImgTraining)
imagenesTest        =   os.listdir(pathImgTest)
universe            =   os.listdir(pathUniverse)

print('Imagenes de training:',len(imagenesTraining), '; Imagenes de test:',len(imagenesTest))
print('Imagenes del universo:',len(universe))

FileNotFoundError: ignored

In [0]:
def parchar(img, tam, paso):
  
  import numpy as np
  
  parches  = []
  dim      = np.shape(img)
  yInicial = 0
  yFinal   = tam
  totalY   = 0
  
  while yFinal <= dim[0]:
    
    xInicial = 0
    xFinal   = tam  
    totalX   = 0
    
    while xFinal <= dim[1]:
      
      parches.append(img[yInicial:yFinal, xInicial:xFinal])
      #print(xInicial, xFinal, yInicial, yFinal)
      
      xInicial +=  paso
      xFinal   +=  paso
      totalX   += 1
    
    yInicial +=  paso
    yFinal   +=  paso
    totalY   +=  1
      
  return parches, totalY,totalX
  
  

In [0]:
def clasificar(parches, etiquetas, tam):
  
  rojos  = []
  verdes = []
  
  for i, etiqueta in enumerate(etiquetas):
    
    if np.sum(etiqueta[:,:,0] == 255) == tam*tam:
      
      rojos.append(parches[i])
      
    if np.sum(etiqueta[:,:,1] == 255) == tam*tam:
      
      verdes.append(parches[i])
      
  return rojos,verdes

In [0]:
def unirDeepFeatures_V2(deep,tam):
    
    features_v = [deep[i].reshape(tam*tam) for i in range(64)]
    pca.fit(features_v)
    final      = pca.transform(features_v)
    descriptor = np.concatenate(final)
    
    return descriptor
  
def unirDeepFeatures_V1(deep,tam):
    
    features_v = [deep[0,:,:,i].reshape(tam*tam) for i in range(64)]
    pca.fit(features_v)
    final      = pca.transform(features_v)
    descriptor = np.concatenate(final)
    
    return descriptor

In [0]:
def blur(img):
  
  dst = cv.GaussianBlur(img,(9,9),0)
  
  return dst

In [0]:
def size(img):
  
  img = cv.resize(img, (int(np.shape(img)[1]*0.4),int(np.shape(img)[0]*0.4)))
  return img

In [0]:
f, axes        =     plt.subplots(3, 2,figsize=(12,15))

for i in range(3):
  
  imagenN = imagenesTraining[i]
  imagen       =     cv.cvtColor(cv.imread(pathImgTraining + imagenN), cv.COLOR_BGR2RGB)
  axes[i,0].axis('off')
  axes[i,0].imshow(imagen) 
  
  label        =     cv.cvtColor(cv.imread(pathLblTraining + imagenN[:-4] + '.png'), cv.COLOR_BGR2RGB)
  axes[i,1].axis('off')
  axes[i,1].imshow(label)

In [0]:
f, axes        =     plt.subplots(3, 2,figsize=(12,15))

for i, imagenN in enumerate(imagenesTest):
  
  imagen       =     cv.cvtColor(cv.imread(pathImgTest + imagenN), cv.COLOR_BGR2RGB)
  axes[i,0].axis('off')
  axes[i,0].imshow(imagen) 
  
  label        =     cv.cvtColor(cv.imread(pathLblTest + imagenN[:-4] + '.png'), cv.COLOR_BGR2RGB)
  axes[i,1].axis('off')
  axes[i,1].imshow(label)

In [0]:
imagenPrueba        = cv.cvtColor(cv.imread(pathImgTest + imagenesTest[1]), cv.COLOR_BGR2RGB)
lblPrueba           = cv.cvtColor(cv.imread(pathLblTest + imagenesTest[1][:-4] + '.png'), cv.COLOR_BGR2RGB) 
imagenPruebaBlur    = blur(imagenPrueba)

plt.figure(figsize=(20,20))

plt.subplot(1,3,1)
plt.axis('off')
plt.imshow(imagenPrueba)

plt.subplot(1,3,2)
plt.axis('off')
plt.imshow(imagenPruebaBlur)

plt.subplot(1,3,3)
plt.axis('off')
plt.imshow(lblPrueba)

In [0]:
imagenPruebaSize    = size(imagenPruebaBlur)
print('Tamaño Original:',np.shape(imagenPrueba), 'Tamaño Reducido',np.shape(imagenPruebaSize))

plt.axis('off')
plt.imshow(imagenPruebaSize)

In [0]:
xTrain = []
yTrain = []


for imagenN in imagenesTraining:
  
  
  imagen   = size(blur(cv.cvtColor(cv.imread(pathImgTraining + imagenN), cv.COLOR_BGR2RGB)))
  labels   = size(cv.cvtColor(cv.imread(pathLblTraining + imagenN[:-4] + '.png'), cv.COLOR_BGR2RGB))
  
  parchesImagen, a, b = parchar(imagen, parchSize, parcheDes)
  parchesLabels, a, b = parchar(labels, parchSize, parcheDes)
  
  rojos, verdes = clasificar(parchesImagen, parchesLabels, parchSize)
  
  for i in tqdm(range(len(rojos))):
    
    parche              = rojos[i]
    array               = np.expand_dims(parche, axis=0)
    caracteristicas     = model_VGG16_conv1.predict(array)
    des                 = unirDeepFeatures_V1(caracteristicas, parchSize)
    xTrain.append(des)
    yTrain.append(0)
    
  for i in tqdm(range(len(verdes))):
    
    parche              = verdes[i]
    array               = np.expand_dims(parche, axis=0)
    caracteristicas     = model_VGG16_conv1.predict(array)
    des                 = unirDeepFeatures_V1(caracteristicas, parchSize)
    xTrain.append(des)
    yTrain.append(1)
    
    
print(np.shape(xTrain), np.shape(yTrain), np.unique(yTrain))    

In [0]:
xTest = []
yTest = []


for imagenN in imagenesTest:
  
  
  imagen   = size(blur(cv.cvtColor(cv.imread(pathImgTest + imagenN), cv.COLOR_BGR2RGB)))
  labels   = size(cv.cvtColor(cv.imread(pathLblTest + imagenN[:-4] + '.png'), cv.COLOR_BGR2RGB))
  
  parchesImagen, a, b = parchar(imagen, parchSize, parcheDes)
  parchesLabels, a, b = parchar(labels, parchSize, parcheDes)
  
  rojos, verdes = clasificar(parchesImagen, parchesLabels, parchSize)
  
  for i in tqdm(range(len(rojos))):
    
    parche              = rojos[i]
    array               = np.expand_dims(parche, axis=0)
    caracteristicas     = model_VGG16_conv1.predict(array)
    des                 = unirDeepFeatures_V1(caracteristicas, parchSize)
    xTest.append(des)
    yTest.append(0)
    
  for i in tqdm(range(len(verdes))):
    
    parche              = verdes[i]
    array               = np.expand_dims(parche, axis=0)
    caracteristicas     = model_VGG16_conv1.predict(array)
    des                 = unirDeepFeatures_V1(caracteristicas, parchSize)
    xTest.append(des)
    yTest.append(1)
    
    
print(np.shape(xTest), np.shape(yTest), np.unique(yTest))    

In [0]:
def show_curve(est, X, y):
    means, stds = [], []
    nfolds_range = range(2,11)
    for nfolds in nfolds_range:
        #print(nfolds)
        s = cross_val_score(est, X, y, cv=KFold(nfolds, shuffle=True), scoring=make_scorer(accuracy_score))
        means.append(np.mean(s))
        stds.append(np.std(s))

    means = np.r_[means]
    stds  = np.r_[stds]
    
    plt.figure(figsize=(8,5))
    plt.plot(nfolds_range, means, label="mean", color="black")
    plt.fill_between(nfolds_range, means-stds, means+stds, color="blue", alpha=.5, label="std")
    plt.xlabel("n folds")
    plt.ylabel("performance")
    plt.legend()

In [0]:
modelo = RandomForestClassifier(criterion = 'entropy', max_depth = 73, n_estimators = 70)
show_curve(modelo, xTrain, yTrain)

In [0]:
modelo.fit(xTrain,yTrain)
print('Test:',modelo.score(xTest,yTest))

In [0]:
def reconstruir(img, modelo, muestreo):
  
  parches, totalY, totalX  =  parchar(img, parchSize, muestreo)
  datos                    =  []
  mapa                     =  np.zeros((totalY, totalX, 3)) 
  indice                   =  0
  
  for i in tqdm(range(len(parches))):
    
    array          = np.expand_dims(parches[i], axis=0)
    features_conv1 = model_VGG16_conv1.predict(array)
    
    datos.append(unirDeepFeatures_V1(features_conv1, parchSize))
  
  clases = modelo.predict(datos)
  
  for y in range(totalY):
    
    for x in range(totalX):
      
      if clases[indice] == 0:
        
        mapa[y,x] = [255,0,0]
      
      else:
        
        mapa[y,x] = [0,255,0] 
        
      indice += 1
      
  return mapa
  

In [0]:
mapa = reconstruir(imagenPruebaSize, modelo, 1)

In [0]:
plt.figure(figsize=(12,12))

plt.subplot(1,2,1)
plt.axis('off') 
plt.title('Reconstruida')
plt.imshow(mapa)

plt.subplot(1,2,2)
plt.axis('off') 
plt.title('Original')
plt.imshow(lblPrueba)